In [1]:
from pyvi.ViTokenizer import tokenize
import re, os, string
import pandas as pd
import math
import numpy as np
import pickle

In [2]:
""" Process corpus and text for BM25 and then load to file for later use: """

# path_to_corpus = 'input\legal_corpus_splitted.csv'
# corpus = pd.read_csv(path_to_corpus)
# corpus = corpus.dropna(subset=['content'])
# corpus.loc[:, 'law_article'] = corpus.apply(lambda row: {'law_id': row['law_title'], 'article_id': str(row['article_id'])}, axis=1) 
# corpus['content'] = corpus['content'].apply(lambda x: remove_stopword(normalize_text(word_segment(clean_text(x)))))
# corpus.to_csv('corpus.csv')

# texts = [
#     [word for word in document.lower().split() if word not in list_stopwords]
#     for document in corpus['content']
# ]
# pickle.dump(texts, "text")

' Process corpus and text for BM25 and then load to file for later use: '

In [4]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()

from sentence_transformers import SentenceTransformer
import cupy as cp
import numpy as np

import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu" 

c:\Users\toant\anaconda3\envs\nlp\Lib\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [9]:
import pandas as pd
train_qna = pd.read_csv("input/train_qna.csv")
bm25_train_qna = pd.read_hdf("bm25_train_qna.h5")
cos_sim_train_qna = pd.read_hdf("cos_sim_train_qna_encode.h5")
train_qna['bm25_query'] = bm25_train_qna['bm25_query']
train_qna['vector'] = cos_sim_train_qna['vector']

In [10]:
qna = train_qna.copy()
qna

,question_id,question,relevant_articles,question_length,num_articles,relevant_titles,bm25_query,vector
0,0637bf82c8b290c7875c5bfddbf91df5,Công an xã xử phạt lỗi không mang bằng lái xe ...,"[{'law_id': '47/2011/tt-bca', 'article_id': '7'}]",14,1,['47/2011/tt-bca'],công_an xã xử_phạt lỗi không mang bằng lái_xe ...,"[-0.2324966, -0.12793744, 0.1934652, -0.052565..."
1,ade2b2ee4f5b869f75f0d183902382af,Phải thực hiện thao tác nạp mẫu vào bình chứa ...,"[{'law_id': '41/2020/tt-bca', 'article_id': '1...",22,1,['41/2020/tt-bca'],phải thực_hiện thao_tác nạp mẫu vào bình chứa ...,"[0.16954486, 0.042344127, 0.10091305, 0.300495..."
2,8fa164edc7a8419cd7dc9ce66d8e695a,Trình tự đánh giá chất lượng hàng năm đối với ...,"[{'law_id': '159/2020/nđ-cp', 'article_id': '1...",19,1,['159/2020/nđ-cp'],trình_tự đánh_giá chất_lượng hàng năm đối_với ...,"[0.13073848, -0.1460132, 0.07958119, 0.1544274..."
3,3aed91309b72097b34427dd28bebd98a,Tử tù chết sau khi tiêm thuốc độc mà không có ...,"[{'law_id': '53/2010/qh12', 'article_id': '60'...",24,2,"['53/2010/qh12', '82/2011/nđ-cp']",tử_tù chết sau khi tiêm thuốc_độc mà không có ...,"[0.19101849, -0.010479315, 0.03856322, 0.04536..."
4,fe9131a8862ce1bfa7de7e2b30eeb39e,Trách nhiệm của Bộ Khoa học và Công nghệ về qu...,"[{'law_id': '63/2020/nđ-cp', 'article_id': '20'}]",25,1,['63/2020/nđ-cp'],trách_nhiệm của bộ khoa_học và công_nghệ về qu...,"[-0.11314676, -0.003094151, 0.046920575, -0.04..."
...,...,...,...,...,...,...,...,...
3191,bd2f979dabdf0033a00ff488b2893e12,Nội dung kiểm tra công tác nghiệm thu công trì...,"[{'law_id': '06/2021/nđ-cp', 'article_id': '24'}]",23,1,['06/2021/nđ-cp'],nội_dung kiểm_tra công_tác nghiệm_thu công_trì...,"[0.30675843, -0.18420339, -0.14305706, -0.0296..."
3192,b7347919f2bd65a2da6f9d0b1728a51c,Mức phạt đối với hành vi chuyển mục đích sử dụ...,"[{'law_id': '35/2019/nđ-cp', 'article_id': '12'}]",34,1,['35/2019/nđ-cp'],mức phạt đối_với hành_vi chuyển mục_đích sử_dụ...,"[-0.05773743, -0.12715302, 0.18285303, -0.0276..."
3193,bda416f28b8c88ce25604a26b01081cb,Trích xuất theo thi hành án hình sự được hiểu ...,"[{'law_id': '41/2019/qh14', 'article_id': '3'}]",12,1,['41/2019/qh14'],trích_xuất theo thi_hành án hình_sự được hiểu ...,"[-0.13125102, -0.05787519, 0.10784773, 0.06275..."
3194,6088765b2e502c7d900a3f9a2f82f2b4,"Chạy ô tô để dầu, nhờn rơi xuống đường phạt ba...","[{'law_id': '100/2019/nđ-cp', 'article_id': '2...",12,1,['100/2019/nđ-cp'],chạy ô_tô để dầu nhờn rơi xuống_đường phạt bao...,"[-0.3036709, -0.2594247, -0.2621314, -0.074477..."


In [11]:
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True, nb_workers=4)

def bm25_func(x):
    from bm25_utilities import get_top_relevance_ids_in_batches 
    import numpy as np 
    a = get_top_relevance_ids_in_batches(x.bm25_query, None, ranking="bm25", unique=True)
    return a

def cos_sim_func(x): 
    from cos_sim_utilities import get_top_relevance_ids_in_batches  
    a = get_top_relevance_ids_in_batches(None, x.vector, ranking="cos_sim", unique=True)
    return a

def rrf_func(x): 
    from rrf_utilities import get_top_relevance_ids_in_batches 
    a = get_top_relevance_ids_in_batches(None, None, ranking="rrf", bm25_ranking=x.bm25, cos_sim_ranking=x.cos_sim, unique=True)
    return a
     

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [ ]:
# run only parts which are necessary

In [12]:
qna["bm25_ranking"] = qna.parallel_apply(bm25_func, axis = 1)

In [13]:
rank = pd.DataFrame() 

In [14]:
rank['bm25'] = qna["bm25_ranking"].copy()

In [15]:
with open("bm25", "wb") as out:
    pickle.dump(rank, out)

In [ ]:
# reset environment to free up memory

In [16]:
qna = train_qna.copy()

In [17]:
qna["cos_sim_ranking"] = qna.progress_apply(cos_sim_func, axis = 1)

In [18]:
rank['cos_sim'] = qna["cos_sim_ranking"].copy()

In [19]:
with open("cos_sim", "wb") as out:
    pickle.dump(rank, out)

In [20]:
del rank

In [ ]:
# reset environment to free up memory

In [21]:
bm25_results = open("bm25", "rb")
cos_sim_results = open("cos_sim", "rb")

rank_1 = pickle.load(bm25_results)
rank_2 = pickle.load(cos_sim_results)

In [22]:
rank_1["cos_sim"] = rank_2["cos_sim"].copy()

In [23]:
qna = qna.filter(["relevant_articles", "relevant_titles"])

In [24]:
# qna["bm25"] = rank_1["bm25"]
# qna["cos_sim"] = rank_1["cos_sim"]

In [25]:
import gc
# del rank_1
del rank_2
del bm25_results
del cos_sim_results
gc.collect()

20

In [26]:
n = 300
list_df = np.array_split(rank_1, math.ceil(len(rank_1)/n))
for i in list_df: 
    i["rr"] = i.parallel_apply(rrf_func, axis = 1) 

c:\Users\toant\anaconda3\envs\nlp\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [37]:
del rank_1

In [38]:
rank = pd.concat(list_df)

In [39]:
del list_df

In [44]:
rank_3 = rank['rr']

In [45]:
with open("complete", "wb") as out:
    pickle.dump(rank_3, out)